<a href="https://colab.research.google.com/github/Deeasan21/AutoEvolv/blob/main/fitness_progress_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -------------------------------
# Skill Improvement Forecaster
# -------------------------------
# Predicts skill level after 4 weeks based on practice/activity inputs
# -------------------------------

import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# 1. Generate Synthetic Dataset

np.random.seed(42)
N = 7000  # Number of samples

# Features (generalized)
current_skill = np.random.uniform(1, 10, size=N)                 # Current skill level (1-10)
practice_time_per_day = np.random.uniform(0, 180, size=N)        # Practice time (0-180 mins/day)
practice_quality = np.random.uniform(1, 10, size=N)              # Practice quality (1-10)
days_per_week = np.random.randint(1, 8, size=N)                  # Practice days per week (1-7)
learning_resources = np.random.uniform(0, 10, size=N)            # Resource richness (0-10)

# Target formula: Skill after 4 weeks (cap between 1 and 10)
noise = np.random.uniform(-0.5, 0.5, size=N)                     # random noise for realism
predicted_skill = (
    current_skill
    + (practice_time_per_day * practice_quality * days_per_week) * 0.002
    + (learning_resources * 0.05)
    + noise
)
predicted_skill = np.clip(predicted_skill, 1, 10)  # Keep within scale

# Features and target
X = np.column_stack([
    current_skill,
    practice_time_per_day,
    practice_quality,
    days_per_week,
    learning_resources
])
y = predicted_skill

print("Sample features:", X[0])
print("Sample target (Predicted Skill):", y[0])

# 2. Normalize Features

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 3. Build Model

tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# 4. Train Model

history = model.fit(X_scaled, y, epochs=100, batch_size=32, verbose=1)
final_loss, final_mae = model.evaluate(X_scaled, y, verbose=0)
print(f"Final training loss: {final_loss:.4f}")
print(f"Final training MAE: {final_mae:.4f} skill points")

# 5. Test with a Sample Input

# Example: [Current Skill, Practice Time, Quality, Days/Week, Resources]
# A dedicated learner, skill 4, 60min/day, quality 8, 5 days/week, resources 7
sample_input = np.array([[4, 60, 8, 5, 7]], dtype=float)
sample_input_scaled = scaler.transform(sample_input)
predicted_skill_4w = model.predict(sample_input_scaled)
print(f"Predicted skill level after 4 weeks: {predicted_skill_4w[0][0]:.2f} (scale 1–10)")

# 6. Save Model

model.save('skill_improvement_forecaster.h5')
print("Model saved as skill_improvement_forecaster.h5")

# 7. (Optional) Convert to TFLite

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('skill_improvement_forecaster.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model converted to TensorFlow Lite (skill_improvement_forecaster.tflite)")


Sample features: [4.37086107 6.98390185 1.65857589 6.         9.66116322]
Sample target (Predicted Skill): 5.292240153533105


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,497 (9.75 KB)

 Trainable params: 2,497 (9.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 31.2825 - mae: 4.5856
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.4793 - mae: 0.9861
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.0357 - mae: 0.8045
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.7274 - mae: 0.6615
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5660 - mae: 0.5821
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4716 - mae: 0.5342
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4056 - mae: 0.4984
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3524 - mae: 0.4674
Epoch 9/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3052 - mae: 0.4361
Epoch 10/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2604 - mae: 0.4034
Epoch 11/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2215 - mae: 0.3726
Epoch 12/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1928 - mae: 0.3459
Epoch 13/100

Predicted skill level after 4 weeks: 8.98 (scale 1–10)
Model saved as skill_improvement_forecaster.h5
Saved artifact at '/tmp/tmpoc_3xip9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 5), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136746177240848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136746177242000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136746177240656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136746177236432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136746177242576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136746177238544: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model converted to TensorFlow Lite (skill_improvement_forecaster.tflite)


In [1]:
# ----------------------------------------------
# Fitness Progress Prediction Model in TensorFlow
# ----------------------------------------------
# 1. Generate synthetic dataset with realistic patterns
# 2. Normalize features
# 3. Build & train regression model
# 4. Test prediction on sample input
# 5. Save model in .h5 format
# 6. (Optional) Convert to TensorFlow Lite
# ----------------------------------------------

import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# ------------- 1. Generate Synthetic Dataset -------------

np.random.seed(42)

N = 7500  # Number of samples

# Feature generation (within realistic fitness ranges)
current_weight = np.random.uniform(100, 250, size=N)
calorie_intake = np.random.uniform(1200, 4000, size=N)
workout_duration = np.round(np.random.uniform(0, 120, size=N))
workout_intensity = np.random.randint(1, 11, size=N)
steps_per_day = np.random.uniform(0, 20000, size=N)

# Calories burned estimation
calories_burned_steps = steps_per_day * 0.04
calories_burned_workout = workout_duration * workout_intensity * 2.0
total_calories_burned = calories_burned_steps + calories_burned_workout

# Caloric deficit (consumed - burned)
caloric_deficit = calorie_intake - total_calories_burned

# Synthetic target: Predicted weight after 4 weeks
noise = np.random.uniform(-2.0, 2.0, size=N)
predicted_weight = (current_weight
                    - 0.001 * caloric_deficit
                    - 0.02 * workout_duration * workout_intensity
                    + noise)

# Features and target
X = np.column_stack([current_weight, calorie_intake, workout_duration, workout_intensity, steps_per_day])
y = predicted_weight

print("Sample features:", X[0])
print("Sample target (predicted weight after 4 weeks):", y[0])

# ------------- 2. Normalize Input Features -------------

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("Feature scaling complete. Example scaled feature:", X_scaled[0])

# ------------- 3. Build TensorFlow Regression Model -------------

tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# ------------- 4. Train Model -------------

history = model.fit(X_scaled, y, epochs=100, batch_size=32, verbose=1)

final_loss, final_mae = model.evaluate(X_scaled, y, verbose=0)
print(f"Final training loss: {final_loss:.4f}")
print(f"Final training MAE: {final_mae:.4f} lbs")

# ------------- 5. Test Model with Sample Input -------------

# Test input: [170, 1900, 50, 6, 7500]
sample_input = np.array([[170, 1900, 50, 6, 7500]], dtype=float)
sample_input_scaled = scaler.transform(sample_input)
predicted_weight_after4w = model.predict(sample_input_scaled)
print(f"Predicted weight after 4 weeks for {sample_input[0]}: {predicted_weight_after4w[0][0]:.2f} lbs")

# ------------- 6. Save Model as H5 -------------

model.save('fitness_progress_model.h5')
print("Model saved as fitness_progress_model.h5")

# ------------- 7. (Optional) Convert to TensorFlow Lite -------------

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('fitness_progress_model.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model converted to TensorFlow Lite (fitness_progress_model.tflite)")

# ------------- End of Script -------------


Sample features: [1.56181018e+02 2.72005877e+03 6.00000000e+01 2.00000000e+00
 1.12524100e+04]
Sample target (predicted weight after 4 weeks): 151.81137607728718
Feature scaling complete. Example scaled feature: [0.37463861 0.54295979 0.5        0.11111111 0.56273661]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,497 (9.75 KB)

 Trainable params: 2,497 (9.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 24491.6953 - mae: 147.3602
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1637.1736 - mae: 32.8556
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 934.2127 - mae: 24.6424
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 309.1308 - mae: 13.8696
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 47.0210 - mae: 5.2574
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 13.1412 - mae: 2.8331
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 7.2558 - mae: 2.1059
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5.3594 - mae: 1.8125
Epoch 9/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.7952 - mae: 1.7211
Epoch 10/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.5889 - mae: 1.6923
Epoch 11/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.4530 - mae: 1.6739
Epoch 12/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.2557 - mae: 1.

Predicted weight after 4 weeks for [1.7e+02 1.9e+03 5.0e+01 6.0e+00 7.5e+03]: 163.16 lbs
Model saved as fitness_progress_model.h5
Saved artifact at '/tmp/tmpdrcz1bh8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 5), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139601566991184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139601566994448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139601566996368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139601566991952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139601566996560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139601566994064: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model converted to TensorFlow Lite (fitness_progress_model.tflite)
